Diamo una prima occhiata al dataset.

In [2]:
import pandas as pd
df_1 = pd.read_csv("./dataset/historical_stock_prices.csv")
df_1

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


Vediamo quanti valori nulli ci sono e quanti valori unici ci sono in date e ticker.

In [3]:
print("date unique {}".format(len(df_1["date"].unique())))
print("ticker unique {}".format(len(df_1["ticker"].unique())))
for col in df_1.columns:
  print(col,df_1[col].isnull().sum())

date unique 12274
ticker unique 5685
ticker 0
open 0
close 0
adj_close 0
low 0
high 0
volume 0
date 0


In [4]:
import pandas as pd
df_2 = pd.read_csv("./dataset/historical_stocks.csv")
df_2

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS
...,...,...,...,...,...
6455,ZOES,NYSE,"ZOE&#39;S KITCHEN, INC.",CONSUMER SERVICES,RESTAURANTS
6456,ZTS,NYSE,ZOETIS INC.,HEALTH CARE,MAJOR PHARMACEUTICALS
6457,ZTO,NYSE,ZTO EXPRESS (CAYMAN) INC.,TRANSPORTATION,TRUCKING FREIGHT/COURIER SERVICES
6458,ZUO,NYSE,"ZUORA, INC.",TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE


Vediamo quanti valori unici ci sono di ticker e sector, notiamo che ci sono dei valori nulli.

In [5]:
print("ticker unique {}".format(len(df_2["ticker"].unique())))
print("sector unique {}".format(len(df_2["sector"].unique())))
for col in df_2.columns:
  print(col,df_2[col].isnull().sum())
print("sector {}".format(df_2["sector"].unique()))

ticker unique 6460
sector unique 14
ticker 0
exchange 0
name 0
sector 1440
industry 1440
sector ['FINANCE' 'CONSUMER SERVICES' 'TECHNOLOGY' 'PUBLIC UTILITIES'
 'CAPITAL GOODS' 'BASIC INDUSTRIES' 'HEALTH CARE' 'CONSUMER DURABLES' nan
 'ENERGY' 'MISCELLANEOUS' 'SECTOR' 'TRANSPORTATION'
 'CONSUMER NON-DURABLES']


Notando che ci sono delle virgole nel campo "name" e per i vari job andremo a separare i campi con tale carettere, delle righe avranno più campi, quindi togliamo tale carettere in tutti i campi.

In [6]:
print("name unique {}".format(len(df_2["name"].unique())))  #Ci sono delle virgole nei nomi quindi rimuoverle cosi da poter fare split con la virgola

name unique 5462


In [7]:
df_2["ticker"] = [str(w).replace(',', ' ') for w in df_2["ticker"]]
df_2["exchange"] = [str(w).replace(',', ' ') for w in df_2["exchange"]]
df_2["name"] = [str(w).replace(',', ' ') for w in df_2["name"]]
df_2["sector"] = [str(w).replace(',', ' ') for w in df_2["sector"]]
df_2["industry"] = [str(w).replace(',', ' ') for w in df_2["industry"]]

In [8]:
print("name unique {}".format(len(df_2["name"].unique())))
print("name unique {}".format((df_2["name"].unique())))

name unique 5462
name unique ['1347 PROPERTY INSURANCE HOLDINGS  INC.' '180 DEGREE CAPITAL CORP.'
 '1-800 FLOWERS.COM  INC.' ... 'ZTO EXPRESS (CAYMAN) INC.' 'ZUORA  INC.'
 'ZYMEWORKS INC.']


In [9]:
df_2

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,1347 PROPERTY INSURANCE HOLDINGS INC.,FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,1347 PROPERTY INSURANCE HOLDINGS INC.,FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,1-800 FLOWERS.COM INC.,CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS
...,...,...,...,...,...
6455,ZOES,NYSE,ZOE&#39;S KITCHEN INC.,CONSUMER SERVICES,RESTAURANTS
6456,ZTS,NYSE,ZOETIS INC.,HEALTH CARE,MAJOR PHARMACEUTICALS
6457,ZTO,NYSE,ZTO EXPRESS (CAYMAN) INC.,TRANSPORTATION,TRUCKING FREIGHT/COURIER SERVICES
6458,ZUO,NYSE,ZUORA INC.,TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE


Poichè il campo "sector" ci servivà per i vari job e contiene dei valori nulli, cerchiamo di inferire il loro valore. L'idea è che siccome un'azienda può avere vari ticker, allora essa può essere su più settori e quindi vi sarà un settore che si ripete più volte. Perciò per ogni ticker (che può essere considerato come un ID) se il valore del campo "sector" associato è nullo, tale valore verrà sostituito con il nome del settore che si ripete più volte per l'azienda considerata.
Esempio : Due ticker "x" ed "y" hanno come nome dell'azienda "a1" e come settore "k", se vi si trova un terzo ticker "z" che ha come nome dell'azienda "a1" ma un valore nullo nel campo del settore, allora vi si inserirà il valore "k", dato che è molto probabile che esso sia il suo valore.


In [10]:
d = df_2.groupby('name')['sector'].apply(list).reset_index()
d

,name,sector
0,1-800 FLOWERS.COM INC.,[CONSUMER SERVICES]
1,1347 PROPERTY INSURANCE HOLDINGS INC.,"[FINANCE, FINANCE]"
2,180 DEGREE CAPITAL CORP.,[FINANCE]
3,1ST CONSTITUTION BANCORP (NJ),[FINANCE]
4,1ST SOURCE CORPORATION,[FINANCE]
...,...,...
5457,ZUMIEZ INC.,[CONSUMER SERVICES]
5458,ZUORA INC.,[TECHNOLOGY]
5459,ZYMEWORKS INC.,[HEALTH CARE]
5460,ZYNERBA PHARMACEUTICALS INC.,[HEALTH CARE]


In [11]:
d["name"]

0                      1-800 FLOWERS.COM  INC.
1       1347 PROPERTY INSURANCE HOLDINGS  INC.
2                     180 DEGREE CAPITAL CORP.
3                1ST CONSTITUTION BANCORP (NJ)
4                       1ST SOURCE CORPORATION
                         ...                  
5457                               ZUMIEZ INC.
5458                               ZUORA  INC.
5459                            ZYMEWORKS INC.
5460             ZYNERBA PHARMACEUTICALS  INC.
5461                                ZYNGA INC.
Name: name, Length: 5462, dtype: object

In [12]:
from collections import defaultdict
def get_max_repeat(l):
  d = defaultdict(int)
  for k in list(l) :
    for j in k:
      d[str(j)] += 1
  d = dict(sorted((value, key) for (key,value) in d.items())) 
  return d.get(max(d.keys()))


In [13]:
for index, name in enumerate(df_2["name"]) :
  if name in list(d["name"]) and  df_2["sector"][index] == "nan" : 
    c = get_max_repeat(d.loc[d['name'] == name]["sector"])
    if c!="nan" :
      df_2["sector"][index] = c

In [14]:
df_2

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,1347 PROPERTY INSURANCE HOLDINGS INC.,FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,1347 PROPERTY INSURANCE HOLDINGS INC.,FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,1-800 FLOWERS.COM INC.,CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS
...,...,...,...,...,...
6455,ZOES,NYSE,ZOE&#39;S KITCHEN INC.,CONSUMER SERVICES,RESTAURANTS
6456,ZTS,NYSE,ZOETIS INC.,HEALTH CARE,MAJOR PHARMACEUTICALS
6457,ZTO,NYSE,ZTO EXPRESS (CAYMAN) INC.,TRANSPORTATION,TRUCKING FREIGHT/COURIER SERVICES
6458,ZUO,NYSE,ZUORA INC.,TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE


In [15]:
print("ticker unique {}".format(len(df_2["ticker"].unique())))
print("sector unique {}".format(len(df_2["sector"].unique())))
for col in df_2.columns:
  print(col,df_2[col].isnull().sum())
print("sector unique {}".format(df_2["sector"].unique()))

ticker unique 6460
sector unique 14
ticker 0
exchange 0
name 0
sector 0
industry 0
sector unique ['FINANCE' 'CONSUMER SERVICES' 'TECHNOLOGY' 'PUBLIC UTILITIES'
 'CAPITAL GOODS' 'BASIC INDUSTRIES' 'HEALTH CARE' 'CONSUMER DURABLES'
 'nan' 'ENERGY' 'MISCELLANEOUS' 'SECTOR' 'TRANSPORTATION'
 'CONSUMER NON-DURABLES']


Da notare che i valori nulli nel campo sector sono diventati "nan", in modo tale che per tecnologia che si userà, basterà individuare il valore "nan" per filtrare i valori nulli.

In [ ]:
df_2.to_csv("./dataset/clean_hs.csv", sep=",")